#### Extending code from zzEnface2Enface.ipybnb

## Populate New Directory, enface_legend

## NOT applying mask yet in the model

In [ ]:
base_dir = r"C:\Users\micah\OneDrive\Gmail OneDrive\Desktop\CMU\_Spring 2025\Projects in BME\Fovea Detection"

import os
import shutil

# === SET YOUR ACTUAL BASE PATH HERE ===
base_dir = r"C:\Users\micah\OneDrive\Gmail OneDrive\Desktop\CMU\_Spring 2025\Projects in BME\Fovea Detection"
src_masked_root = os.path.join(base_dir, "enface_images", "img_masked_img")
src_raw_root = os.path.join(base_dir, "predictions")
dst_root = os.path.join(base_dir, "enface_legend")

# === Ensure destination root exists ===
os.makedirs(dst_root, exist_ok=True)

# === Patient folders ===
patient_folders = sorted([d for d in os.listdir(src_masked_root) if os.path.isdir(os.path.join(src_masked_root, d))])

# === Copy logic ===
for folder in patient_folders:
    dst_folder = os.path.join(dst_root, folder)
    dst_masked = os.path.join(dst_folder, "enface mask")
    dst_raw = os.path.join(dst_folder, "raw enface")

    os.makedirs(dst_masked, exist_ok=True)
    os.makedirs(dst_raw, exist_ok=True)

    # Get enface PNG from masked folder
    src_masked_folder = os.path.join(src_masked_root, folder)
    enface_png = next((f for f in os.listdir(src_masked_folder) if "enface" in f.lower() and f.lower().endswith(".png")), None)
    if enface_png:
        shutil.copy(os.path.join(src_masked_folder, enface_png), os.path.join(dst_masked, enface_png))
    else:
        print(f"❌ No enface PNG found in {src_masked_folder}")

    # Get JPG from predictions folder
    src_raw_folder = os.path.join(src_raw_root, folder)
    if os.path.exists(src_raw_folder):
        raw_jpg = next((f for f in os.listdir(src_raw_folder) if f.lower().endswith(".jpg")), None)
        if raw_jpg:
            shutil.copy(os.path.join(src_raw_folder, raw_jpg), os.path.join(dst_raw, raw_jpg))
        else:
            print(f"❌ No JPG in {src_raw_folder}")
    else:
        print(f"⚠️ Predictions folder missing for {folder}")

print("✅ enface_legend structure created and populated.")


### Filter out poor enface images

In [ ]:
import os
import shutil

# === Updated ignore list ===
ignored_folders = [
    "1002_right", "1003_left", "1004_left", "1005_right", "1006_right",
    "1007_left", "1009_right", "1011_right", "1012_left", "1013_left",
    "1013_right", "1014_left", "1014_right", "1015_right", "1016_left",
    "1016_right", "1017_right", "1019_right", "1021_right", "1022_left",
    "1023_left", "1023_right", "1024_right", "1025_left", "1026_left",
    "1027_left", "1027_right", "1028_left", "1029_left", "1029_right",
    "1030_left", "1030_right", "1031_right"
]

# === Paths ===
base_dir = r"C:\Users\micah\OneDrive\Gmail OneDrive\Desktop\CMU\_Spring 2025\Projects in BME\Fovea Detection"
src_legend_root = os.path.join(base_dir, "enface_legend")
dst_curated_root = os.path.join(base_dir, "curated_enface")

# === Recreate curated_enface directory ===
if os.path.exists(dst_curated_root):
    shutil.rmtree(dst_curated_root)
os.makedirs(dst_curated_root, exist_ok=True)

# === Copy allowed folders ===
for folder in os.listdir(src_legend_root):
    src_path = os.path.join(src_legend_root, folder)
    dst_path = os.path.join(dst_curated_root, folder)
    if os.path.isdir(src_path) and folder not in ignored_folders:
        shutil.copytree(src_path, dst_path)

# === Summary ===
print("✅ curated_enface recreated successfully.")
print(f"Ignored {len(ignored_folders)} folders:")
print(ignored_folders)


# ViT based MAEncoder
### see newest code. we are now using relative positional embeddings. not using multi-cropping.

In [ ]:


# import os
# import torch
# import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms
# from PIL import Image
# import numpy as np
# import random

# THRESH = 0.75

# # =============== Dataset Loader with Full Augmentations ===============
# class EnfaceMaskedDataset(Dataset):
#     def __init__(self, root_dir, image_size=512):
#         self.root_dir = root_dir
#         self.image_size = image_size
#         self.patients = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
#         self.augment = transforms.Compose([
#             transforms.RandomAffine(degrees=15, translate=(0.2, 0.2), scale=(0.8, 1.2)),
#             transforms.RandomHorizontalFlip(),
#             transforms.RandomVerticalFlip()
#         ])
#         self.to_tensor = transforms.ToTensor()
#         self.resize = transforms.Resize((image_size, image_size))

#     def __len__(self):
#         return len(self.patients)

#     def __getitem__(self, idx):
#         patient = self.patients[idx]
#         enface_path = os.path.join(self.root_dir, patient, 'raw enface')
#         mask_path = os.path.join(self.root_dir, patient, 'enface mask')

#         enface_file = next((f for f in os.listdir(enface_path) if f.lower().endswith('.jpg')), None)
#         mask_file = next((f for f in os.listdir(mask_path) if f.lower().endswith(('.png', '.jpg'))), None)

#         enface_img = Image.open(os.path.join(enface_path, enface_file)).convert('RGB')
#         mask_img = Image.open(os.path.join(mask_path, mask_file)).convert('L')

#         seed = np.random.randint(2147483647)
#         random.seed(seed)
#         torch.manual_seed(seed)
#         enface_img = self.augment(enface_img)
#         random.seed(seed)
#         torch.manual_seed(seed)
#         mask_img = self.augment(mask_img)

#         enface_tensor = self.to_tensor(self.resize(enface_img)) * 2 - 1
#         mask_tensor = self.to_tensor(self.resize(mask_img)).squeeze(0)
#         mask_tensor = (mask_tensor > THRESH).float()

#         return enface_tensor, mask_tensor

# # =============== Relative Positional Bias ===============
# def get_rel_pos_index(h, w):
#     coords = torch.stack(torch.meshgrid(torch.arange(h), torch.arange(w), indexing="ij"))
#     coords_flat = coords.flatten(1)
#     rel_coords = coords_flat[:, :, None] - coords_flat[:, None, :]
#     rel_coords = rel_coords.permute(1, 2, 0).contiguous()
#     rel_coords[:, :, 0] += h - 1
#     rel_coords[:, :, 1] += w - 1
#     rel_coords[:, :, 0] *= 2 * w - 1
#     return rel_coords.sum(-1)

# # =============== Vision Transformer Encoder (Relative) ===============
# class PatchEmbedding(nn.Module):
#     def __init__(self, in_channels=3, patch_size=16, emb_size=768):
#         super().__init__()
#         self.patch_size = patch_size
#         self.proj = nn.Conv2d(in_channels, emb_size, kernel_size=patch_size, stride=patch_size)
#         self.cls_token = nn.Parameter(torch.randn(1, 1, emb_size))

#     def forward(self, x):
#         B = x.shape[0]
#         x = self.proj(x).flatten(2).transpose(1, 2)  # [B, N, D]
#         cls_tokens = self.cls_token.expand(B, -1, -1)
#         x = torch.cat((cls_tokens, x), dim=1)
#         return x

# class RelPosAttention(nn.Module):
#     def __init__(self, dim, num_heads, window_size):
#         super().__init__()
#         self.num_heads = num_heads
#         self.qkv = nn.Linear(dim, dim * 3, bias=True)
#         self.scale = (dim // num_heads) ** -0.5
#         self.proj = nn.Linear(dim, dim)
#         self.rel_index = get_rel_pos_index(window_size, window_size)
#         self.rel_bias = nn.Parameter(torch.zeros((2 * window_size - 1) ** 2, num_heads))

#     def forward(self, x):
#         B, N, C = x.shape
#         qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
#         q, k, v = qkv[0], qkv[1], qkv[2]
#         attn = (q @ k.transpose(-2, -1)) * self.scale
#         rel = self.rel_bias[self.rel_index.to(x.device)].unsqueeze(0).permute(0, 3, 1, 2)
#         attn = attn + rel[:, :, :N, :N]
#         attn = attn.softmax(dim=-1)
#         x = (attn @ v).transpose(1, 2).reshape(B, N, C)
#         return self.proj(x)

# class TransformerBlock(nn.Module):
#     def __init__(self, dim, num_heads, mlp_ratio=4.0, dropout=0.1, window_size=32):
#         super().__init__()
#         self.norm1 = nn.LayerNorm(dim)
#         self.attn = RelPosAttention(dim, num_heads, window_size)
#         self.norm2 = nn.LayerNorm(dim)
#         self.mlp = nn.Sequential(
#             nn.Linear(dim, int(dim * mlp_ratio)),
#             nn.GELU(),
#             nn.Dropout(dropout),
#             nn.Linear(int(dim * mlp_ratio), dim),
#             nn.Dropout(dropout)
#         )

#     def forward(self, x):
#         x = x + self.attn(self.norm1(x))
#         x = x + self.mlp(self.norm2(x))
#         return x

# class ViTEncoder(nn.Module):
#     def __init__(self, patch_size=16, emb_size=768, img_size=512, num_layers=12, num_heads=12):
#         super().__init__()
#         self.patch_embed = PatchEmbedding(patch_size=patch_size, emb_size=emb_size)
#         self.layers = nn.Sequential(*[
#             TransformerBlock(dim=emb_size, num_heads=num_heads, window_size=img_size // patch_size)
#             for _ in range(num_layers)
#         ])

#     def forward(self, x):
#         x = self.patch_embed(x)[:, 1:, :]
#         return self.layers(x)

# # =============== Run Encoder on Curated Data ===============
# if __name__ == '__main__':
#     # curated_path = r"C:\Users\micah\OneDrive\Gmail OneDrive\Desktop\CMU\_Spring 2025\Projects in BME\Fovea Detection\curated_enface"
#     # curated_path = "/content/drive/MyDrive/curted_enface"
#     dataset = EnfaceMaskedDataset(curated_path, image_size=512)
#     loader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)

#     model = ViTEncoder(img_size=512)
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)
#     model.train()

#     optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.05)
#     patchify = nn.Unfold(kernel_size=16, stride=16)

#     for epoch in range(200):
#         epoch_loss = 0
#         for imgs, _ in loader:
#             imgs = imgs.to(device)
#             full_img = imgs.clone()
#             masked_full = full_img.clone()

#             # Full image masking (random patch occlusion)
#             B, C, H, W = masked_full.shape
#             for b in range(B):
#                 for _ in range(20):
#                     top = random.randint(0, H - 64)
#                     left = random.randint(0, W - 64)
#                     masked_full[b, :, top:top+64, left:left+64] = 0

#             # Multi-crop masking
#             loss_total = 0
#             stride = 128
#             crop_size = 256
#             for i in range(0, H - crop_size + 1, stride):
#                 for j in range(0, W - crop_size + 1, stride):
#                     crop = imgs[:, :, i:i+crop_size, j:j+crop_size].clone()
#                     masked_crop = crop.clone()
#                     for b in range(B):
#                         top = random.randint(0, crop_size - 64)
#                         left = random.randint(0, crop_size - 64)
#                         masked_crop[b, :, top:top+64, left:left+64] = 0

#                     patch_crops = patchify(crop).transpose(1, 2)
#                     patch_masked = patchify(masked_crop).transpose(1, 2)
#                     missing_patch = (patch_masked == 0).any(dim=2)
#                     valid_mask = ~missing_patch

#                     features = model(masked_crop)
#                     valid_pred = features[valid_mask].reshape(-1, features.size(-1))
#                     valid_targets = patch_crops[valid_mask].reshape(-1, patch_crops.size(-1))
#                     loss_total += nn.MSELoss()(valid_pred, valid_targets)

#             # Full-image masking loss
#             full_patch = patchify(full_img).transpose(1, 2)
#             full_patch_mask = patchify(masked_full).transpose(1, 2)
#             missing_patch_full = (full_patch_mask == 0).any(dim=2)
#             valid_mask_full = ~missing_patch_full
#             full_features = model(masked_full)
#             valid_pred_full = full_features[valid_mask_full].reshape(-1, full_features.size(-1))
#             valid_targets_full = full_patch[valid_mask_full].reshape(-1, full_patch.size(-1))
#             loss_total += nn.MSELoss()(valid_pred_full, valid_targets_full)

#             optimizer.zero_grad()
#             loss_total.backward()
#             optimizer.step()

#             epoch_loss += loss_total.item()

#         print(f"[Epoch {epoch}] Combined Loss: {epoch_loss:.4f}")

#     # os.makedirs("models", exist_ok=True)
#     # torch.save(model.state_dict(), "models/vit_encoder_multicrop_augmented.pth")
#     # save_path = "/content/drive/MyDrive/curted_enface/supreme_vit_mae.pth"
#     os.makedirs(os.path.dirname(save_path), exist_ok=True)
#     torch.save(model.state_dict(), save_path)
#     print("✅ Encoder saved at:", save_path)

# ran this on colab

# Ensure that the masks are loading properly
### did not add morphological closing or contrast

In [ ]:
# masked_autoencoder_vit.py

import os
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

THRESH_COLOR1 = (255, 200, 0)  # Bright orange (outer mask)
THRESH_COLOR2 = (255, 255, 0)  # Lighter yellow-orange (inner mask)
TOLERANCE = 60  # Allow variation in RGB for thresholding

# =============== Dataset Loader ===============
class EnfaceMaskedDataset(Dataset):
    def __init__(self, root_dir, image_size=512):
        self.root_dir = root_dir
        self.image_size = image_size
        self.patients = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.augment = transforms.Compose([
            transforms.RandomAffine(degrees=15, translate=(0.2, 0.2), scale=(0.8, 1.2)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip()
        ])
        self.to_tensor = transforms.ToTensor()
        self.resize = transforms.Resize((image_size, image_size))

    def __len__(self):
        return len(self.patients)

    def __getitem__(self, idx):
        patient = self.patients[idx]
        enface_path = os.path.join(self.root_dir, patient, 'raw enface')
        mask_path = os.path.join(self.root_dir, patient, 'enface mask')

        enface_file = next(f for f in os.listdir(enface_path) if f.lower().endswith('.jpg'))
        mask_file = next(f for f in os.listdir(mask_path) if f.lower().endswith(('.png', '.jpg')))

        enface_img = Image.open(os.path.join(enface_path, enface_file)).convert('RGB')
        mask_img = Image.open(os.path.join(mask_path, mask_file)).convert('RGB')

        seed = np.random.randint(2147483647)
        random.seed(seed)
        torch.manual_seed(seed)
        enface_img = self.augment(enface_img)
        random.seed(seed)
        torch.manual_seed(seed)
        mask_img = self.augment(mask_img)

        enface_tensor = self.to_tensor(self.resize(enface_img)) * 2 - 1

        mask_np = np.array(self.resize(mask_img))

        lower1 = np.array([c - TOLERANCE for c in THRESH_COLOR1])
        upper1 = np.array([c + TOLERANCE for c in THRESH_COLOR1])
        lower2 = np.array([c - TOLERANCE for c in THRESH_COLOR2])
        upper2 = np.array([c + TOLERANCE for c in THRESH_COLOR2])

        mask_bin1 = np.all((mask_np >= lower1) & (mask_np <= upper1), axis=-1)
        mask_bin2 = np.all((mask_np >= lower2) & (mask_np <= upper2), axis=-1)
        mask_combined = (mask_bin1 | mask_bin2).astype(np.float32)
        mask_tensor = torch.tensor(mask_combined)

        return enface_tensor, mask_tensor

# =============== Visualize Random Transformed Samples ===============
def visualize_augmented_samples(dataset_path, num_samples=9):
    dataset = EnfaceMaskedDataset(dataset_path)
    indices = random.sample(range(len(dataset)), num_samples)

    plt.figure(figsize=(12, 12))
    for i, idx in enumerate(indices):
        img, mask = dataset[idx]
        masked_img = img.clone()
        for c in range(3):
            masked_img[c] *= (1 - mask)

        img_np = ((masked_img + 1) / 2).permute(1, 2, 0).numpy()
        mask_np = mask.numpy()

        plt.subplot(3, 3, i + 1)
        plt.imshow(img_np)
        plt.imshow(mask_np, cmap="Reds", alpha=0.4)
        plt.axis("off")
        plt.title(f"Sample {idx}")

    plt.tight_layout()
    plt.show()

if __name__ == '__main__':
    visualize_augmented_samples("curated_enface")

# UMapp++ With Skip Connections Based Decoder

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from PIL import Image
import numpy as np
import random
import matplotlib.pyplot as plt

# Define threshold colors for segmentation masks
THRESH_COLOR1 = (255, 200, 0)
THRESH_COLOR2 = (255, 255, 0)
TOLERANCE = 60

# =============== Utility: 2D Sin-Cos Positional Embedding ===============
def get_2d_sincos_pos_embed(embed_dim, grid_size):
    """Generate 2D positional embeddings with correct dimensions"""
    assert embed_dim % 2 == 0, "Embedding dimension must be even"
    # Generate grid coordinates for both axes
    grid_h = np.arange(grid_size, dtype=np.float32)
    grid_w = np.arange(grid_size, dtype=np.float32)
    
    # Get embeddings for each axis
    emb_h = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid_h)
    emb_w = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid_w)
    
    # Create 2D embeddings through broadcasting
    emb = np.concatenate([
        np.repeat(emb_h[:, None, :], grid_size, axis=1),
        np.repeat(emb_w[None, :, :], grid_size, axis=0)
    ], axis=-1)
    
    return emb.reshape(-1, embed_dim)

def get_1d_sincos_pos_embed_from_grid(embed_dim, pos):
    """Generate 1D positional embeddings from grid positions"""
    assert embed_dim % 2 == 0
    omega = np.arange(embed_dim // 2, dtype=np.float32)
    omega /= embed_dim / 2.
    omega = 1. / 10000**omega  # Frequency calculation
    
    # Ensure pos is treated as array even with single value
    pos = np.asarray(pos).reshape(-1)
    out = np.outer(pos, omega)  # Vectorized calculation
    
    return np.concatenate([np.sin(out), np.cos(out)], axis=1)

# =============== Dataset Loader for Segmentation ===============
class EnfaceSegmentationDataset(Dataset):
    def __init__(self, root_dir, image_size=512):
        self.root_dir = root_dir
        self.image_size = image_size
        self.patients = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.augment = transforms.Compose([
            transforms.RandomAffine(degrees=15, translate=(0.2, 0.2), scale=(0.8, 1.2)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip()
        ])
        self.to_tensor = transforms.ToTensor()
        self.resize = transforms.Resize((image_size, image_size))

    def __len__(self):
        return len(self.patients)

    def __getitem__(self, idx):
        patient = self.patients[idx]
        enface_path = os.path.join(self.root_dir, patient, 'raw enface')
        mask_path = os.path.join(self.root_dir, patient, 'enface mask')

        # Handle potential file not found errors gracefully
        try:
            enface_file = next(f for f in os.listdir(enface_path) if f.lower().endswith('.jpg'))
            mask_file = next(f for f in os.listdir(mask_path) if f.lower().endswith(('.png', '.jpg')))
        except (StopIteration, FileNotFoundError):
            print(f"Error finding image files for patient: {patient}")
            # Return a valid but empty sample as fallback
            return torch.zeros(3, self.image_size, self.image_size), torch.zeros(self.image_size, self.image_size)

        try:
            enface_img = Image.open(os.path.join(enface_path, enface_file)).convert('RGB')
            mask_img = Image.open(os.path.join(mask_path, mask_file)).convert('RGB')
        except Exception as e:
            print(f"Error opening image files for patient {patient}: {e}")
            return torch.zeros(3, self.image_size, self.image_size), torch.zeros(self.image_size, self.image_size)

        # Apply consistent augmentation to both image and mask
        seed = np.random.randint(2147483647)
        random.seed(seed)
        torch.manual_seed(seed)
        enface_img = self.augment(enface_img)
        random.seed(seed)
        torch.manual_seed(seed)
        mask_img = self.augment(mask_img)

        # Convert to tensor and normalize to [-1, 1] for the image
        enface_tensor = self.to_tensor(self.resize(enface_img)) * 2 - 1

        # Create binary mask by thresholding for both color ranges
        mask_np = np.array(self.resize(mask_img))
        lower1 = np.array([c - TOLERANCE for c in THRESH_COLOR1])
        upper1 = np.array([c + TOLERANCE for c in THRESH_COLOR1])
        lower2 = np.array([c - TOLERANCE for c in THRESH_COLOR2])
        upper2 = np.array([c + TOLERANCE for c in THRESH_COLOR2])

        mask_bin1 = np.all((mask_np >= lower1) & (mask_np <= upper1), axis=-1)
        mask_bin2 = np.all((mask_np >= lower2) & (mask_np <= upper2), axis=-1)
        mask_combined = (mask_bin1 | mask_bin2).astype(np.float32)
        mask_tensor = torch.tensor(mask_combined)

        return enface_tensor, mask_tensor


# =============== Relative Positional Embeddings ===============
def get_rel_pos_index(h, w):
    """Calculate relative position indices"""
    coords = torch.stack(torch.meshgrid(torch.arange(h), torch.arange(w), indexing="ij"))
    coords_flat = coords.flatten(1)
    rel_coords = coords_flat[:, :, None] - coords_flat[:, None, :]
    rel_coords = rel_coords.permute(1, 2, 0).contiguous()
    rel_coords[:, :, 0] += h - 1
    rel_coords[:, :, 1] += w - 1
    rel_coords[:, :, 0] *= 2 * w - 1
    return rel_coords.sum(-1)

class RelativeAttention(nn.Module):
    """Attention mechanism with relative positional embeddings"""
    def __init__(self, dim, num_heads, grid_size):
        super().__init__()
        self.num_heads = num_heads
        self.scale = (dim // num_heads) ** -0.5
        self.qkv = nn.Linear(dim, dim * 3, bias=False)
        self.proj = nn.Linear(dim, dim)

        self.rel_index = get_rel_pos_index(grid_size, grid_size)
        self.rel_bias = nn.Parameter(torch.zeros((2 * grid_size - 1) ** 2, num_heads))
        nn.init.trunc_normal_(self.rel_bias, std=0.02)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        # Calculate attention scores
        attn = (q @ k.transpose(-2, -1)) * self.scale
        
        # Handle relative position bias
        rel_bias = self.rel_bias[self.rel_index.to(x.device)]
        if N > self.rel_index.shape[0]:
            # Handle potential mismatch in sequence length
            pad_size = N - self.rel_index.shape[0]
            rel_bias = F.pad(rel_bias, (0, 0, 0, pad_size, 0, pad_size), value=0)
        rel = rel_bias.unsqueeze(0).permute(0, 3, 1, 2)
        
        attn = attn + rel[:, :, :N, :N]
        attn = attn.softmax(dim=-1)
        out = (attn @ v).transpose(1, 2).reshape(B, N, C)
        return self.proj(out)

class Block(nn.Module):
    """Transformer block with relative positional embeddings"""
    def __init__(self, dim, num_heads, grid_size, mlp_ratio=4., norm_layer=nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = RelativeAttention(dim, num_heads, grid_size)
        self.norm2 = norm_layer(dim)
        self.mlp = nn.Sequential(
            nn.Linear(dim, int(dim * mlp_ratio)),
            nn.GELU(),
            nn.Linear(int(dim * mlp_ratio), dim),
        )

    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x

# =============== MAE Model (Encoder Only) ===============
class MAEEncoder(nn.Module):
    """MAE Encoder with relative positional embeddings"""
    def __init__(
        self,
        img_size=512,
        patch_size=32,
        in_chans=3,
        embed_dim=768,
        depth=8,
        num_heads=12,
        mlp_ratio=4.,
        norm_layer=nn.LayerNorm,
    ):
        super().__init__()
        # Encoder parameters
        self.patch_size = patch_size
        self.in_chans = in_chans
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        
        # Size info
        self.img_size = img_size
        self.patch_embed = nn.Conv2d(
            in_chans, embed_dim, kernel_size=patch_size, stride=patch_size
        )
        self.grid_size = img_size // patch_size
        self.num_patches = self.grid_size ** 2
        
        # Encoder embeddings
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(
            torch.zeros(1, 1 + self.num_patches, embed_dim),
            requires_grad=False
        )  # fixed sin-cos embedding

        # Encoder blocks with intermediate outputs for skip connections
        self.blocks = nn.ModuleList([
            Block(
                embed_dim, 
                num_heads, 
                self.grid_size, 
                mlp_ratio=mlp_ratio, 
                norm_layer=norm_layer
            )
            for _ in range(depth)
        ])
        
        self.norm = norm_layer(embed_dim)
        
        # Initialize weights
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize cls_token
        nn.init.normal_(self.cls_token, std=0.02)
        
        # Initialize positional embedding
        pos_embed = get_2d_sincos_pos_embed(self.embed_dim, self.grid_size)
        pos_embed = torch.from_numpy(pos_embed).float().unsqueeze(0)
        pe_token = torch.zeros(1, 1, self.embed_dim)
        self.pos_embed.data.copy_(torch.cat([pe_token, pos_embed], dim=1))
        
        # Initialize linear layers
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.LayerNorm):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x, return_intermediates=False):
        """
        Forward pass with option to return intermediate features for skip connections
        """
        # Convert image to patches
        x = self.patch_embed(x)  # [B, C, H/P, W/P]
        x = x.flatten(2).transpose(1, 2)  # [B, N, C]
        
        # Add positional embedding (without cls token)
        x = x + self.pos_embed[:, 1:, :]
        
        # Append cls token
        cls_token = self.cls_token + self.pos_embed[:, :1, :]
        cls_tokens = cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        
        # Apply transformer blocks and collect intermediate features
        intermediates = []
        for blk in self.blocks:
            x = blk(x)
            if return_intermediates:
                intermediates.append(x[:, 1:].clone())  # Skip cls token
        
        # Apply final normalization
        x = self.norm(x)
        
        # Return features after cls token
        if return_intermediates:
            return x[:, 1:], intermediates  # Return without cls token
        else:
            return x[:, 1:]  # Return without cls token

# =============== UNet++ Decoder with Skip Connections ===============
class ConvBlock(nn.Module):
    """Double convolution block for UNet++"""
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if mid_channels is None:
            mid_channels = out_channels
            
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        return self.double_conv(x)

class UpBlock(nn.Module):
    def __init__(self, in_channels, out_channels, skip_channels=0, bilinear=False):
        super().__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            up_channels = in_channels
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            up_channels = in_channels // 2

        self.conv = ConvBlock(up_channels + skip_channels, out_channels)

    def forward(self, x1, x2=None):
        x1 = self.up(x1)
        if x2 is not None:
            # Handle potential size mismatch
            diffY = x2.size()[2] - x1.size()[2]
            diffX = x2.size()[3] - x1.size()[3]
            x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2, diffY // 2, diffY - diffY // 2])
            x = torch.cat([x2, x1], dim=1)
        else:
            x = x1
        return self.conv(x)


class UNetPPDecoder(nn.Module):
    """UNet++ decoder for segmentation with dense skip connections"""
    def __init__(self, encoder_dim=768, grid_size=16):
        super().__init__()
        # Reshaping from encoder output to 2D feature maps
        self.grid_size = grid_size
        self.encoder_dim = encoder_dim
        
        # Nested U-Net structure with dense skip connections
        # X_0,0 (encoder output) -> X_0,1 -> X_0,2 -> X_0,3 -> X_0,4
        # X_1,0                  -> X_1,1 -> X_1,2 -> X_1,3
        # X_2,0                  -> X_2,1 -> X_2,2
        # X_3,0                  -> X_3,1
        # X_4,0
        
        # First level (deepest) 
        self.conv_x00 = ConvBlock(encoder_dim, 512)
        
        # Second level
        self.up_x01 = UpBlock(512, 256)
        self.conv_x10 = ConvBlock(encoder_dim, 256)
        self.conv_x11 = ConvBlock(512, 256)  # 256 + 256
        
        # Third level
        self.up_x02 = UpBlock(256, 128)
        self.up_x12 = UpBlock(256, 128)
        self.conv_x20 = ConvBlock(encoder_dim, 128)
        self.conv_x21 = ConvBlock(256, 128)  # 128 + 128
        self.conv_x22 = ConvBlock(384, 128)  # 128 + 128 + 128
        
        # Fourth level
        self.up_x03 = UpBlock(128, 64)
        self.up_x13 = UpBlock(128, 64)
        self.up_x23 = UpBlock(128, 64)
        self.conv_x30 = ConvBlock(encoder_dim, 64)
        self.conv_x31 = ConvBlock(128, 64)   # 64 + 64
        self.conv_x32 = ConvBlock(192, 64)   # 64 + 64 + 64
        self.conv_x33 = ConvBlock(256, 64)   # 64 + 64 + 64 + 64
        
        # Final level (nearest to output)
        self.up_x04 = UpBlock(64, 32)
        self.up_x14 = UpBlock(64, 32)
        self.up_x24 = UpBlock(64, 32)
        self.up_x34 = UpBlock(64, 32)
        self.conv_x40 = ConvBlock(encoder_dim, 32)
        self.conv_x41 = ConvBlock(64, 32)    # 32 + 32
        self.conv_x42 = ConvBlock(96, 32)    # 32 + 32 + 32
        self.conv_x43 = ConvBlock(128, 32)   # 32 + 32 + 32 + 32
        self.conv_x44 = ConvBlock(160, 32)   # 32 + 32 + 32 + 32 + 32
        
        # Output layers
        self.final = nn.Sequential(
            nn.Conv2d(32, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.Conv2d(16, 1, kernel_size=1)
        )
        
    def forward(self, encoded_features, intermediates):
        B = encoded_features.shape[0]
        grid_size = self.grid_size
        
        # Process encoder features 
        x00 = encoded_features.permute(0, 2, 1).reshape(B, self.encoder_dim, grid_size, grid_size)
        x00 = self.conv_x00(x00)  # [B, 512, 16, 16]

        # Process intermediates with upsampling
        if len(intermediates) >= 4:
            # Upsample encoder intermediates to match decoder levels
            x10 = F.interpolate(intermediates[3].permute(0, 2, 1).reshape(B, self.encoder_dim, grid_size, grid_size), 
                            scale_factor=2, mode='bilinear')
            x10 = self.conv_x10(x10)  # [B, 256, 32, 32]

            x20 = F.interpolate(intermediates[2].permute(0, 2, 1).reshape(B, self.encoder_dim, grid_size, grid_size),
                            scale_factor=4, mode='bilinear')
            x20 = self.conv_x20(x20)  # [B, 128, 64, 64]

            x30 = F.interpolate(intermediates[1].permute(0, 2, 1).reshape(B, self.encoder_dim, grid_size, grid_size),
                            scale_factor=8, mode='bilinear')
            x30 = self.conv_x30(x30)  # [B, 64, 128, 128]

            x40 = F.interpolate(intermediates[0].permute(0, 2, 1).reshape(B, self.encoder_dim, grid_size, grid_size),
                            scale_factor=16, mode='bilinear')
            x40 = self.conv_x40(x40)  # [B, 32, 256, 256]

        
        # First level up path and skip connections
        x01 = self.up_x01(x00)  # [B, 256, 32, 32]
        x11 = self.conv_x11(torch.cat([x10, x01], dim=1))  # [B, 256, 32, 32]
        
        # Second level
        x02 = self.up_x02(x01)  # [B, 128, 64, 64]
        x12 = self.up_x12(x11)  # [B, 128, 64, 64]
        x22 = self.conv_x22(torch.cat([x20, x02, x12], dim=1))  # [B, 128, 64, 64]
        
        # Third level
        x03 = self.up_x03(x02)  # [B, 64, 128, 128]
        x13 = self.up_x13(x12)  # [B, 64, 128, 128]
        x23 = self.up_x23(x22)  # [B, 64, 128, 128]
        x33 = self.conv_x33(torch.cat([x30, x03, x13, x23], dim=1))  # [B, 64, 128, 128]
        
        # Fourth level
        x04 = self.up_x04(x03)  # [B, 32, 256, 256]
        x14 = self.up_x14(x13)  # [B, 32, 256, 256]
        x24 = self.up_x24(x23)  # [B, 32, 256, 256]
        x34 = self.up_x34(x33)  # [B, 32, 256, 256]
        x44 = self.conv_x44(torch.cat([x40, x04, x14, x24, x34], dim=1))  # [B, 32, 256, 256]
        
        # Final upsampling to reach original image size
        # Need to go from 256x256 to 512x512
        final_features = F.interpolate(x44, scale_factor=2, mode='bilinear', align_corners=True)
        
        # Output layer - produces segmentation mask
        out = self.final(final_features)  # [B, 1, 512, 512]
        
        return out

# =============== Combined Model ===============
class MAESegmentation(nn.Module):
    """Combined MAE encoder with UNet++ decoder for segmentation"""
    def __init__(
        self,
        img_size=512,
        patch_size=32,
        in_chans=3,
        embed_dim=768,
        encoder_depth=8,
        encoder_num_heads=12,
        mlp_ratio=4.,
        pretrained_mae_path=None,
        freeze_encoder=True
    ):
        super().__init__()
        
        # Create encoder with exact same architecture as MAE encoder
        self.encoder = MAEEncoder(
            img_size=img_size,
            patch_size=patch_size,
            in_chans=in_chans,
            embed_dim=embed_dim,
            depth=encoder_depth,
            num_heads=encoder_num_heads,
            mlp_ratio=mlp_ratio
        )
        
        # Load pretrained weights if provided
        if pretrained_mae_path:
            self._load_pretrained_encoder(pretrained_mae_path)
        
        # Freeze encoder parameters if specified
        if freeze_encoder:
            for param in self.encoder.parameters():
                param.requires_grad = False
                
        # Calculate grid size based on image size and patch size
        grid_size = img_size // patch_size
        
        # Create UNet++ decoder
        self.decoder = UNetPPDecoder(
            encoder_dim=embed_dim, 
            grid_size=grid_size
        )
    
    def _load_pretrained_encoder(self, pretrained_path):
        """Load pretrained weights from MAE model checkpoint"""
        try:
            # Handle different checkpoint formats
            checkpoint = torch.load(pretrained_path, map_location='cpu')
            
            # If the checkpoint is a dict with 'model_state_dict' key
            if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
                state_dict = checkpoint['model_state_dict']
            else:
                state_dict = checkpoint
                
            # Filter and load encoder-specific parameters
            encoder_dict = {}
            for k, v in state_dict.items():
                # Only load parameters for components present in our encoder
                # Remove decoder related keys and other non-encoder components
                if k.startswith('patch_embed') or k.startswith('blocks') or k.startswith('norm') or \
                   k.startswith('cls_token') or k.startswith('pos_embed'):
                    encoder_dict[k] = v
            
            # Attempt to load weights
            msg = self.encoder.load_state_dict(encoder_dict, strict=False)
            print(f"Loaded pretrained MAE encoder from {pretrained_path}")
            print(f"Missing keys: {msg.missing_keys}")
            print(f"Unexpected keys: {msg.unexpected_keys}")
        except Exception as e:
            print(f"Error loading pretrained weights: {e}")
    
    def forward(self, x):
        # Get encoder features with intermediates for skip connections
        encoded_features, intermediates = self.encoder(x, return_intermediates=True)
        
        # Pass to decoder
        segmentation = self.decoder(encoded_features, intermediates)
        
        return segmentation

# =============== Loss Functions ===============
# =============== Loss Functions ===============
def dice_loss(pred, target, smooth=1e-6):
    """Dice loss for segmentation"""
    pred = torch.sigmoid(pred)
    pred = pred.view(-1)
    target = target.view(-1)
    intersection = (pred * target).sum()
    dice = (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)
    return 1 - dice

def area_loss(pred, max_area=40*40):
    """Penalize predictions that exceed the maximum allowed area"""
    # Convert logits to binary predictions
    pred_binary = (torch.sigmoid(pred) > 0.5).float()
    # Calculate area of each prediction in the batch
    areas = pred_binary.sum(dim=[2, 3])  # Sum over height and width dimensions
    # Calculate penalty for exceeding max_area
    # Use ReLU to only penalize when area > max_area
    penalty = F.relu(areas - max_area) / max_area
    return penalty.mean()

def empty_penalty(pred, weight=0.1):
    """Penalize empty predictions (no segmentation)"""
    pred_binary = (torch.sigmoid(pred) > 0.5).float()
    # Check if any prediction is completely empty
    empty_mask = (pred_binary.sum(dim=[2, 3]) == 0).float()
    return weight * empty_mask.mean()

def combined_segmentation_loss(pred, target, area_weight=0.2, empty_weight=0.1, max_area=75*75):
    """Combined loss function for segmentation with area constraint"""
    # Binary cross entropy loss
    bce = F.binary_cross_entropy_with_logits(pred, target.unsqueeze(1))
    
    # Dice loss
    dice = dice_loss(pred, target.unsqueeze(1))
    
    # Area constraint loss
    area = area_loss(pred, max_area=max_area)
    
    # Empty prediction penalty
    empty = empty_penalty(pred, weight=empty_weight)
    
    # Combine losses (weighted sum)
    total_loss = 0.4 * bce + 0.4 * dice + area_weight * area + empty_weight * empty
    
    # Return individual loss components for monitoring
    return total_loss, {
        'bce': bce.item(),
        'dice': dice.item(),
        'area': area.item(),
        'empty': empty.item()
    }


# =============== Training Function ===============
def train_segmentation(
    data_path,
    pretrained_mae_path,
    batch_size=2,
    num_epochs=150,
    learning_rate=1.5e-4,  # Match MAE's learning rate
    output_dir="segmentation_results2",
    save_interval=5,
    device=None,
    max_area=75*75,  # Maximum area in pixels for fovea segmentation
    area_weight=0.2,  # Weight for area constraint loss
    empty_weight=0.1  # Weight for empty prediction penalty
):
    """Train segmentation model using pretrained MAE encoder with area constraints"""
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, "checkpoints"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "visuals"), exist_ok=True)
    
    # Load dataset
    dataset = EnfaceSegmentationDataset(data_path)
    dataloader = DataLoader(
        dataset, 
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,  # Set to 0 to avoid CUDA issues
        pin_memory=True if torch.cuda.is_available() else False
    )
    print(f"Dataset loaded with {len(dataset)} samples")
    
    # Create model
    model = MAESegmentation(
        img_size=512,
        patch_size=32,  # Match MAE's patch size
        embed_dim=768,  # Match MAE's embedding dimension
        encoder_depth=8,  # Match MAE's encoder depth
        encoder_num_heads=12,  # Match MAE's encoder heads
        pretrained_mae_path=pretrained_mae_path,
        freeze_encoder=True  # Freeze encoder weights initially
    ).to(device)
    
    # Optimizer - only update decoder parameters if encoder is frozen
    optimizer = torch.optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=learning_rate,
        betas=(0.9, 0.95),  # Match MAE's optimizer settings
        weight_decay=0.05   # Match MAE's weight decay
    )

    # Learning rate scheduler - use CosineAnnealingLR to match MAE's scheduler
    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=num_epochs,
        eta_min=1e-5
    )

    best_loss = float('inf')
    
    # Create CSV file to track loss components
    loss_tracker_path = os.path.join(output_dir, "loss_tracking.csv")
    with open(loss_tracker_path, 'w') as f:
        f.write("epoch,batch,total_loss,bce_loss,dice_loss,area_loss,empty_loss\n")
    
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        epoch_bce_loss = 0.0
        epoch_dice_loss = 0.0
        epoch_area_loss = 0.0
        epoch_empty_loss = 0.0
        num_batches = 0

        for batch_idx, (images, masks) in enumerate(dataloader):
            images = images.to(device)
            masks = masks.to(device)

            # Forward pass
            outputs = model(images)
            loss, loss_components = combined_segmentation_loss(
                outputs, 
                masks, 
                area_weight=area_weight,
                empty_weight=empty_weight,
                max_area=max_area
            )

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            
            # Clip gradients for stability (like in MAE)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()

            # Track metrics
            batch_loss = loss.item()
            epoch_loss += batch_loss
            epoch_bce_loss += loss_components['bce']
            epoch_dice_loss += loss_components['dice']
            epoch_area_loss += loss_components['area']
            epoch_empty_loss += loss_components['empty']
            num_batches += 1

            # Print progress and log loss components
            if batch_idx % 10 == 0:
                print(f"Epoch {epoch}/{num_epochs-1}, Batch {batch_idx}/{len(dataloader)}, " +
                     f"Loss: {batch_loss:.4f}, BCE: {loss_components['bce']:.4f}, " +
                     f"Dice: {loss_components['dice']:.4f}, Area: {loss_components['area']:.4f}, " + 
                     f"Empty: {loss_components['empty']:.4f}")
                
                # Log to CSV
                with open(loss_tracker_path, 'a') as f:
                    f.write(f"{epoch},{batch_idx},{batch_loss:.6f},{loss_components['bce']:.6f}," +
                           f"{loss_components['dice']:.6f},{loss_components['area']:.6f},{loss_components['empty']:.6f}\n")

        # Update learning rate
        lr_scheduler.step()
        
        # End of epoch tracking
        avg_epoch_loss = epoch_loss / num_batches if num_batches > 0 else 0
        avg_bce_loss = epoch_bce_loss / num_batches if num_batches > 0 else 0
        avg_dice_loss = epoch_dice_loss / num_batches if num_batches > 0 else 0
        avg_area_loss = epoch_area_loss / num_batches if num_batches > 0 else 0
        avg_empty_loss = epoch_empty_loss / num_batches if num_batches > 0 else 0
        
        print(f"Epoch {epoch} Summary:")
        print(f"  Total Loss: {avg_epoch_loss:.4f}")
        print(f"  BCE Loss: {avg_bce_loss:.4f}")
        print(f"  Dice Loss: {avg_dice_loss:.4f}")
        print(f"  Area Loss: {avg_area_loss:.4f}")
        print(f"  Empty Loss: {avg_empty_loss:.4f}")
        print(f"  Learning Rate: {lr_scheduler.get_last_lr()[0]:.6f}")

        # Save model checkpoint
        if epoch % save_interval == 0 or epoch == num_epochs - 1 or avg_epoch_loss < best_loss:
            checkpoint_path = os.path.join(output_dir, "checkpoints", f"model_epoch_{epoch}.pt")
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': {
                    'total': avg_epoch_loss,
                    'bce': avg_bce_loss,
                    'dice': avg_dice_loss,
                    'area': avg_area_loss,
                    'empty': avg_empty_loss
                },
            }, checkpoint_path)
            print(f"Model saved to {checkpoint_path}")
            
            if avg_epoch_loss < best_loss:
                best_loss = avg_epoch_loss
                best_model_path = os.path.join(output_dir, "checkpoints", "best_model.pt")
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': {
                        'total': avg_epoch_loss,
                        'bce': avg_bce_loss,
                        'dice': avg_dice_loss,
                        'area': avg_area_loss,
                        'empty': avg_empty_loss
                    },
                }, best_model_path)
                print(f"✅ New best model found: {best_model_path}")

        # Save visualizations and area stats
        if epoch % save_interval == 0 or epoch == num_epochs - 1:
            vis_dir = os.path.join(output_dir, "visuals", f"epoch_{epoch}")
            os.makedirs(vis_dir, exist_ok=True)
            
            model.eval()
            with torch.no_grad():
                # Get a batch of samples
                images, masks = next(iter(dataloader))
                images = images.to(device)
                pred_masks_logits = model(images)
                pred_masks = (torch.sigmoid(pred_masks_logits) > 0.5).float()
                
                # Calculate areas for visualization
                areas = []
                for i in range(min(4, len(images))):
                    # Original image
                    save_image((images[i] + 1) / 2, os.path.join(vis_dir, f"input_{i}.png"))
                    
                    # Ground truth mask
                    gt_mask = masks[i].unsqueeze(0)
                    gt_area = gt_mask.sum().item()
                    save_image(gt_mask, os.path.join(vis_dir, f"gt_mask_{i}.png"))
                    
                    # Predicted mask
                    pred = pred_masks[i]
                    pred_area = pred.sum().item()
                    save_image(pred, os.path.join(vis_dir, f"pred_mask_{i}.png"))
                    
                    # Overlay of prediction on input
                    overlay_img = images[i].cpu().clone()
                    # Set the red channel to highlight the prediction
                    overlay_img[0] = overlay_img[0] * (1 - pred[0]) + pred[0]
                    save_image((overlay_img + 1) / 2, os.path.join(vis_dir, f"overlay_{i}.png"))
                    
                    areas.append((i, gt_area, pred_area))
                
                # Save area stats
                with open(os.path.join(vis_dir, "area_stats.txt"), 'w') as f:
                    f.write("Sample, GT Area (pixels), Pred Area (pixels), Max Area (75x75 = 5625)\n")
                    for i, gt_area, pred_area in areas:
                        f.write(f"{i}, {gt_area}, {pred_area}, 5625\n")
    
    print("Training complete!")
    return model

train_segmentation('curated_enface', 'models/mae_model_epoch_199.pt')

# Visualizations

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import random

# === Utility Functions ===
def get_2d_sincos_pos_embed(embed_dim, grid_size):
    """Generate 2D positional embeddings with correct dimensions"""
    assert embed_dim % 2 == 0, "Embedding dimension must be even"
    # Generate grid coordinates for both axes
    grid_h = np.arange(grid_size, dtype=np.float32)
    grid_w = np.arange(grid_size, dtype=np.float32)
    
    # Get embeddings for each axis
    emb_h = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid_h)
    emb_w = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid_w)
    
    # Create 2D embeddings through broadcasting
    emb = np.concatenate([
        np.repeat(emb_h[:, None, :], grid_size, axis=1),
        np.repeat(emb_w[None, :, :], grid_size, axis=0)
    ], axis=-1)
    
    return emb.reshape(-1, embed_dim)

def get_1d_sincos_pos_embed_from_grid(embed_dim, pos):
    """Generate 1D positional embeddings from grid positions"""
    assert embed_dim % 2 == 0
    omega = np.arange(embed_dim // 2, dtype=np.float32)
    omega /= embed_dim / 2.
    omega = 1. / 10000**omega  # Frequency calculation
    
    # Ensure pos is treated as array even with single value
    pos = np.asarray(pos).reshape(-1)
    out = np.outer(pos, omega)  # Vectorized calculation
    
    return np.concatenate([np.sin(out), np.cos(out)], axis=1)

def get_rel_pos_index(h, w):
    """Calculate relative position indices"""
    coords = torch.stack(torch.meshgrid(torch.arange(h), torch.arange(w), indexing="ij"))
    coords_flat = coords.flatten(1)
    rel_coords = coords_flat[:, :, None] - coords_flat[:, None, :]
    rel_coords = rel_coords.permute(1, 2, 0).contiguous()
    rel_coords[:, :, 0] += h - 1
    rel_coords[:, :, 1] += w - 1
    rel_coords[:, :, 0] *= 2 * w - 1
    return rel_coords.sum(-1)

# === Overlay Utility ===
def overlay_mask(image, mask, alpha=0.4):
    mask_bin = (mask > 0.8).astype(np.float32)
    overlay = image.astype(np.float32).copy()
    overlay[mask_bin > 0] = overlay[mask_bin > 0] * (1 - alpha) + alpha * 255
    return overlay.clip(0, 255).astype(np.uint8)

# === RelativeAttention for MAE Architecture ===
class RelativeAttention(nn.Module):
    """Attention mechanism with relative positional embeddings"""
    def __init__(self, dim, num_heads, grid_size):
        super().__init__()
        self.num_heads = num_heads
        self.scale = (dim // num_heads) ** -0.5
        self.qkv = nn.Linear(dim, dim * 3, bias=False)
        self.proj = nn.Linear(dim, dim)

        self.rel_index = get_rel_pos_index(grid_size, grid_size)
        self.rel_bias = nn.Parameter(torch.zeros((2 * grid_size - 1) ** 2, num_heads))
        nn.init.trunc_normal_(self.rel_bias, std=0.02)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        # Calculate attention scores
        attn = (q @ k.transpose(-2, -1)) * self.scale
        
        # Handle relative position bias
        rel_bias = self.rel_bias[self.rel_index.to(x.device)]
        if N > self.rel_index.shape[0]:
            # Handle potential mismatch in sequence length
            pad_size = N - self.rel_index.shape[0]
            rel_bias = F.pad(rel_bias, (0, 0, 0, pad_size, 0, pad_size), value=0)
        rel = rel_bias.unsqueeze(0).permute(0, 3, 1, 2)
        
        attn = attn + rel[:, :, :N, :N]
        attn = attn.softmax(dim=-1)
        out = (attn @ v).transpose(1, 2).reshape(B, N, C)
        return self.proj(out)

class Block(nn.Module):
    """Transformer block with relative positional embeddings"""
    def __init__(self, dim, num_heads, grid_size, mlp_ratio=4., norm_layer=nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = RelativeAttention(dim, num_heads, grid_size)
        self.norm2 = norm_layer(dim)
        self.mlp = nn.Sequential(
            nn.Linear(dim, int(dim * mlp_ratio)),
            nn.GELU(),
            nn.Linear(int(dim * mlp_ratio), dim),
        )

    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x

# === MAE Encoder ===
class MAEEncoder(nn.Module):
    """MAE Encoder with relative positional embeddings"""
    def __init__(
        self,
        img_size=512,
        patch_size=32,
        in_chans=3,
        embed_dim=768,
        depth=8,
        num_heads=12,
        mlp_ratio=4.,
        norm_layer=nn.LayerNorm,
    ):
        super().__init__()
        # Encoder parameters
        self.patch_size = patch_size
        self.in_chans = in_chans
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        
        # Size info
        self.img_size = img_size
        self.patch_embed = nn.Conv2d(
            in_chans, embed_dim, kernel_size=patch_size, stride=patch_size
        )
        self.grid_size = img_size // patch_size
        self.num_patches = self.grid_size ** 2
        
        # Encoder embeddings
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(
            torch.zeros(1, 1 + self.num_patches, embed_dim),
            requires_grad=False
        )  # fixed sin-cos embedding

        # Encoder blocks with intermediate outputs for skip connections
        self.blocks = nn.ModuleList([
            Block(
                embed_dim, 
                num_heads, 
                self.grid_size, 
                mlp_ratio=mlp_ratio, 
                norm_layer=norm_layer
            )
            for _ in range(depth)
        ])
        
        self.norm = norm_layer(embed_dim)
        
        # Initialize weights
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize cls_token
        nn.init.normal_(self.cls_token, std=0.02)
        
        # Initialize positional embedding
        pos_embed = get_2d_sincos_pos_embed(self.embed_dim, self.grid_size)
        pos_embed = torch.from_numpy(pos_embed).float().unsqueeze(0)
        pe_token = torch.zeros(1, 1, self.embed_dim)
        self.pos_embed.data.copy_(torch.cat([pe_token, pos_embed], dim=1))
        
        # Initialize linear layers
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.LayerNorm):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x, return_intermediates=False):
        """
        Forward pass with option to return intermediate features for skip connections
        """
        # Convert image to patches
        x = self.patch_embed(x)  # [B, C, H/P, W/P]
        x = x.flatten(2).transpose(1, 2)  # [B, N, C]
        
        # Add positional embedding (without cls token)
        x = x + self.pos_embed[:, 1:, :]
        
        # Append cls token
        cls_token = self.cls_token + self.pos_embed[:, :1, :]
        cls_tokens = cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        
        # Apply transformer blocks and collect intermediate features
        intermediates = []
        for blk in self.blocks:
            x = blk(x)
            if return_intermediates:
                intermediates.append(x[:, 1:].clone())  # Skip cls token
        
        # Apply final normalization
        x = self.norm(x)
        
        # Return features after cls token
        if return_intermediates:
            return x[:, 1:], intermediates  # Return without cls token
        else:
            return x[:, 1:]  # Return without cls token

# === UNet++ Decoder ===
class ConvBlock(nn.Module):
    """Double convolution block for UNet++"""
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if mid_channels is None:
            mid_channels = out_channels
            
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        return self.double_conv(x)

class UpBlock(nn.Module):
    def __init__(self, in_channels, out_channels, skip_channels=0, bilinear=False):
        super().__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            up_channels = in_channels
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            up_channels = in_channels // 2

        self.conv = ConvBlock(up_channels + skip_channels, out_channels)

    def forward(self, x1, x2=None):
        x1 = self.up(x1)
        if x2 is not None:
            # Handle potential size mismatch
            diffY = x2.size()[2] - x1.size()[2]
            diffX = x2.size()[3] - x1.size()[3]
            x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2, diffY // 2, diffY - diffY // 2])
            x = torch.cat([x2, x1], dim=1)
        else:
            x = x1
        return self.conv(x)

class UNetPPDecoder(nn.Module):
    """UNet++ decoder for segmentation with dense skip connections"""
    def __init__(self, encoder_dim=768, grid_size=16):
        super().__init__()
        # Reshaping from encoder output to 2D feature maps
        self.grid_size = grid_size
        self.encoder_dim = encoder_dim
        
        # Nested U-Net structure with dense skip connections
        # First level (deepest) 
        self.conv_x00 = ConvBlock(encoder_dim, 512)
        
        # Second level
        self.up_x01 = UpBlock(512, 256)
        self.conv_x10 = ConvBlock(encoder_dim, 256)
        self.conv_x11 = ConvBlock(512, 256)  # 256 + 256
        
        # Third level
        self.up_x02 = UpBlock(256, 128)
        self.up_x12 = UpBlock(256, 128)
        self.conv_x20 = ConvBlock(encoder_dim, 128)
        self.conv_x21 = ConvBlock(256, 128)  # 128 + 128
        self.conv_x22 = ConvBlock(384, 128)  # 128 + 128 + 128
        
        # Fourth level
        self.up_x03 = UpBlock(128, 64)
        self.up_x13 = UpBlock(128, 64)
        self.up_x23 = UpBlock(128, 64)
        self.conv_x30 = ConvBlock(encoder_dim, 64)
        self.conv_x31 = ConvBlock(128, 64)   # 64 + 64
        self.conv_x32 = ConvBlock(192, 64)   # 64 + 64 + 64
        self.conv_x33 = ConvBlock(256, 64)   # 64 + 64 + 64 + 64
        
        # Final level (nearest to output)
        self.up_x04 = UpBlock(64, 32)
        self.up_x14 = UpBlock(64, 32)
        self.up_x24 = UpBlock(64, 32)
        self.up_x34 = UpBlock(64, 32)
        self.conv_x40 = ConvBlock(encoder_dim, 32)
        self.conv_x41 = ConvBlock(64, 32)    # 32 + 32
        self.conv_x42 = ConvBlock(96, 32)    # 32 + 32 + 32
        self.conv_x43 = ConvBlock(128, 32)   # 32 + 32 + 32 + 32
        self.conv_x44 = ConvBlock(160, 32)   # 32 + 32 + 32 + 32 + 32
        
        # Output layers
        self.final = nn.Sequential(
            nn.Conv2d(32, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.Conv2d(16, 1, kernel_size=1)
        )
        
    def forward(self, encoded_features, intermediates):
        B = encoded_features.shape[0]
        grid_size = self.grid_size
        
        # Process encoder features 
        x00 = encoded_features.permute(0, 2, 1).reshape(B, self.encoder_dim, grid_size, grid_size)
        x00 = self.conv_x00(x00)  # [B, 512, 16, 16]

        # Process intermediates with upsampling
        if len(intermediates) >= 4:
            # Upsample encoder intermediates to match decoder levels
            x10 = F.interpolate(intermediates[3].permute(0, 2, 1).reshape(B, self.encoder_dim, grid_size, grid_size), 
                            scale_factor=2, mode='bilinear')
            x10 = self.conv_x10(x10)  # [B, 256, 32, 32]

            x20 = F.interpolate(intermediates[2].permute(0, 2, 1).reshape(B, self.encoder_dim, grid_size, grid_size),
                            scale_factor=4, mode='bilinear')
            x20 = self.conv_x20(x20)  # [B, 128, 64, 64]

            x30 = F.interpolate(intermediates[1].permute(0, 2, 1).reshape(B, self.encoder_dim, grid_size, grid_size),
                            scale_factor=8, mode='bilinear')
            x30 = self.conv_x30(x30)  # [B, 64, 128, 128]

            x40 = F.interpolate(intermediates[0].permute(0, 2, 1).reshape(B, self.encoder_dim, grid_size, grid_size),
                            scale_factor=16, mode='bilinear')
            x40 = self.conv_x40(x40)  # [B, 32, 256, 256]

        
        # First level up path and skip connections
        x01 = self.up_x01(x00)  # [B, 256, 32, 32]
        x11 = self.conv_x11(torch.cat([x10, x01], dim=1))  # [B, 256, 32, 32]
        
        # Second level
        x02 = self.up_x02(x01)  # [B, 128, 64, 64]
        x12 = self.up_x12(x11)  # [B, 128, 64, 64]
        x22 = self.conv_x22(torch.cat([x20, x02, x12], dim=1))  # [B, 128, 64, 64]
        
        # Third level
        x03 = self.up_x03(x02)  # [B, 64, 128, 128]
        x13 = self.up_x13(x12)  # [B, 64, 128, 128]
        x23 = self.up_x23(x22)  # [B, 64, 128, 128]
        x33 = self.conv_x33(torch.cat([x30, x03, x13, x23], dim=1))  # [B, 64, 128, 128]
        
        # Fourth level
        x04 = self.up_x04(x03)  # [B, 32, 256, 256]
        x14 = self.up_x14(x13)  # [B, 32, 256, 256]
        x24 = self.up_x24(x23)  # [B, 32, 256, 256]
        x34 = self.up_x34(x33)  # [B, 32, 256, 256]
        x44 = self.conv_x44(torch.cat([x40, x04, x14, x24, x34], dim=1))  # [B, 32, 256, 256]
        
        # Final upsampling to reach original image size
        # Need to go from 256x256 to 512x512
        final_features = F.interpolate(x44, scale_factor=2, mode='bilinear', align_corners=True)
        
        # Output layer - produces segmentation mask
        out = self.final(final_features)  # [B, 1, 512, 512]
        
        return out

# === Complete Model ===
class MAESegmentation(nn.Module):
    """Combined MAE encoder with UNet++ decoder for segmentation"""
    def __init__(
        self,
        img_size=512,
        patch_size=32,
        in_chans=3,
        embed_dim=768,
        encoder_depth=8,
        encoder_num_heads=12,
        mlp_ratio=4.,
    ):
        super().__init__()
        
        # Create encoder
        self.encoder = MAEEncoder(
            img_size=img_size,
            patch_size=patch_size,
            in_chans=in_chans,
            embed_dim=embed_dim,
            depth=encoder_depth,
            num_heads=encoder_num_heads,
            mlp_ratio=mlp_ratio
        )
                
        # Calculate grid size based on image size and patch size
        grid_size = img_size // patch_size
        
        # Create UNet++ decoder
        self.decoder = UNetPPDecoder(
            encoder_dim=embed_dim, 
            grid_size=grid_size
        )
    
    def forward(self, x):
        # Get encoder features with intermediates for skip connections
        encoded_features, intermediates = self.encoder(x, return_intermediates=True)
        
        # Pass to decoder
        segmentation = self.decoder(encoded_features, intermediates)
        
        return segmentation

# ============== MAIN CODE FOR VISUALIZATION ==============
print("Setting up environment...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the trained model
print("Loading model...")
model = MAESegmentation(
    img_size=512,
    patch_size=32,
    embed_dim=768,
    encoder_depth=8,
    encoder_num_heads=12
).to(device)

# First load the encoder weights
try:
    print("Loading encoder...")
    encoder_path = "models/mae_model_epoch_199.pt"
    encoder_state = torch.load(encoder_path, map_location=device)
    model.encoder.load_state_dict(encoder_state, strict=False)
    print("Encoder loaded successfully")
except Exception as e:
    print(f"Error loading encoder: {e}")
    print("Using randomly initialized encoder")

# Then load the whole model with pretrained decoder
try:
    print("Loading decoder...")
    decoder_path = "segmentation_results2/checkpoints/best_model.pt"
    checkpoint = torch.load(decoder_path, map_location=device)
    
    if isinstance(checkpoint, dict) and "model_state_dict" in checkpoint:
        print("Found model_state_dict in checkpoint")
        model.load_state_dict(checkpoint["model_state_dict"], strict=False)
    else:
        print("Loading state_dict directly")
        model.load_state_dict(checkpoint, strict=False)
        
    print("Model loaded successfully")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Using randomly initialized weights")

model.eval()

# Set up image transformation
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])

# Load images
prediction_root = "C:/Users/micah/OneDrive/Gmail OneDrive/Desktop/CMU/_Spring 2025/Projects in BME/Fovea Detection/predictions"
print(f"Loading images from {prediction_root}")

# Find folders
folders = []
try:
    folders = sorted([os.path.join(prediction_root, f) for f in os.listdir(prediction_root) 
                     if os.path.isdir(os.path.join(prediction_root, f))])
    # Randomly shuffle for different visualizations each run
    random.shuffle(folders)
    print(f"Found {len(folders)} image folders")
except Exception as e:
    print(f"Error reading folders: {e}")
    folders = []

# Process images and get predictions
imgs, overlays, filenames = [], [], []
for folder in folders[:20]:  # Limit to 20 folders for efficiency
    try:
        img_file = next((f for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.png', '.jpeg'))), None)
        if not img_file:
            continue

        img_path = os.path.join(folder, img_file)
        print(f"Processing {img_file}")
        
        # Load and preprocess image
        img = Image.open(img_path).convert("RGB")
        img_tensor = transform(img).unsqueeze(0).to(device) * 2 - 1  # Scale to [-1, 1]

        # Optional: Apply fovea mask if available
        mask_path = os.path.join(folder, "mask", "fovea_mask.png")
        if os.path.exists(mask_path):
            mask_img = Image.open(mask_path).convert("L")
            mask_tensor = transform(mask_img).squeeze(0).to(device)
            for c in range(3):
                img_tensor[0, c] *= (1 - mask_tensor)
            print(f"[✓] Applied mask for {img_file}")

        # Generate prediction
        with torch.no_grad():
            pred = model(img_tensor)
            pred_sigmoid = torch.sigmoid(pred)[0, 0].cpu().numpy()
            print(f"Prediction range: {pred_sigmoid.min():.3f} to {pred_sigmoid.max():.3f}")

        # Convert image to grayscale for visualization
        img_np = np.array(img.resize((512, 512)))
        img_gray = np.mean(img_np, axis=2) if img_np.ndim == 3 else img_np
        
        # Create overlay with different thresholds
        threshold = 0.5



        ##########################################################################################

        mask_binary = (pred_sigmoid > threshold).astype(np.float32)
        overlay = overlay_mask(img_gray, mask_binary, alpha=0.4)

        # Store results
        imgs.append(img_gray)
        overlays.append(overlay)
        filenames.append(img_file)
        
        if len(imgs) >= 16:  # Limit to 16 images for display
            break
    except Exception as e:
        print(f"Error processing {folder}: {e}")
        continue

# If we couldn't load any real images, create mock data
if len(imgs) == 0:
    print("No images processed. Creating mock data for visualization...")
    from skimage.draw import circle
    
    # Create mock retinal images and fovea masks
    for i in range(16):
        # Create circular retinal image with fovea
        mock_img = np.zeros((512, 512), dtype=np.uint8)
        # Retina background
        rr, cc = circle(256, 256, 230, shape=(512, 512))
        mock_img[rr, cc] = 180
        
        # Fovea region
        fovea_x = random.randint(226, 286)
        fovea_y = random.randint(226, 286)
        fovea_r = random.randint(15, 30)
        rr, cc = circle(fovea_y, fovea_x, fovea_r, shape=(512, 512))
        
        # Create binary mask and overlay
        mask = np.zeros((512, 512), dtype=np.float32)
        mask[rr, cc] = 1.0
        overlay = overlay_mask(mock_img, mask)
        
        # Add to our collections
        imgs.append(mock_img)
        overlays.append(overlay)
        filenames.append(f"mock_image_{i}.jpg")

print(f"Successfully processed {len(imgs)} images")

# === Create visualizations ===

# 1. Single random image with side-by-side comparison
random_idx = random.randint(0, len(imgs)-1)
f, ax = plt.subplots(1, 2, figsize=(12, 5))
ax[0].imshow(imgs[random_idx], cmap='gray')
ax[0].set_title(f"Input Image: {filenames[random_idx]}", fontsize=14)
ax[0].axis('off')
ax[1].imshow(overlays[random_idx], cmap='gray')
ax[1].set_title("Predicted Fovea Mask Overlay", fontsize=14)
ax[1].axis('off')
plt.tight_layout()
plt.savefig('random_fovea_prediction.png')
plt.show()

# 2. Multiple side-by-side comparisons
num_samples = min(3, len(imgs))
sample_indices = random.sample(range(len(imgs)), num_samples)

f, axs = plt.subplots(num_samples, 2, figsize=(12, 5*num_samples))
for i, idx in enumerate(sample_indices):
    if num_samples == 1:
        ax_row = axs
    else:
        ax_row = axs[i]
        
    ax_row[0].imshow(imgs[idx], cmap='gray')
    ax_row[0].set_title(f"Input: {filenames[idx]}", fontsize=14)
    ax_row[0].axis('off')
    
    ax_row[1].imshow(overlays[idx], cmap='gray')
    ax_row[1].set_title(f"Fovea Detection", fontsize=14)
    ax_row[1].axis('off')

plt.tight_layout()
plt.savefig('multiple_fovea_predictions.png')
plt.show()

# 3. Grid view of all detections
grid_size = min(16, len(imgs))
grid_indices = random.sample(range(len(imgs)), grid_size)

rows = int(np.ceil(np.sqrt(grid_size)))
cols = int(np.ceil(grid_size / rows))

f, axs = plt.subplots(rows, cols, figsize=(16, 16))
axs = np.atleast_2d(axs)  # Ensure 2D even with single row/column

for i, idx in enumerate(grid_indices):
    row, col = i // cols, i % cols
    axs[row, col].imshow(overlays[idx], cmap='gray')
    axs[row, col].set_title(f"{filenames[idx]}", fontsize=10)
    axs[row, col].axis('off')

# Turn off any unused subplots
for i in range(grid_size, rows * cols):
    row, col = i // cols, i % cols
    axs[row, col].axis('off')

plt.tight_layout()
plt.savefig('grid_predictions.png')
plt.show()

# 4. Threshold comparison for a single image
random_idx = random.randint(0, len(imgs)-1)
img = imgs[random_idx]


##################s

import os
import random
import torch
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from torchvision import transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Load Model (Assume already defined and trained) ===
model.eval()

# === Paths ===
enface_root = "enface_legend"
subfolders = [os.path.join(enface_root, d) for d in os.listdir(enface_root) if os.path.isdir(os.path.join(enface_root, d))]

transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])

def overlay_mask(image, mask, alpha=0.4):
    mask_bin = (mask > 0.5).astype(np.float32)
    overlay = image.astype(np.float32).copy()
    overlay[mask_bin > 0] = overlay[mask_bin > 0] * (1 - alpha) + alpha * 255
    return overlay.clip(0, 255).astype(np.uint8)

def get_valid_samples(num_needed=24):
    samples = []
    random.shuffle(subfolders)
    for folder in subfolders:
        raw_path = os.path.join(folder, "raw enface")
        mask_path = os.path.join(folder, "enface mask")

        try:
            img_file = next(f for f in os.listdir(raw_path) if f.lower().endswith(('.jpg', '.jpeg', '.png')))
            mask_file = next(f for f in os.listdir(mask_path) if f.lower().endswith(('.jpg', '.jpeg', '.png')))
        except StopIteration:
            continue

        img_path = os.path.join(raw_path, img_file)
        mask_path = os.path.join(mask_path, mask_file)

        if os.path.exists(img_path) and os.path.exists(mask_path):
            samples.append((img_path, mask_path))
        if len(samples) >= num_needed:
            break
    return samples

# === Load Samples ===
samples = get_valid_samples(24)

# === Display in 4 batches of 6x3 ===
for batch in range(4):
    fig, axs = plt.subplots(6, 3, figsize=(15, 20))
    for row in range(6):
        idx = batch * 6 + row
        if idx >= len(samples):
            for col in range(3):
                axs[row, col].axis("off")
            continue

        img_path, mask_path = samples[idx]
        try:
            img = Image.open(img_path).convert("RGB")
            mask = Image.open(mask_path).convert("L")
        except:
            for col in range(3):
                axs[row, col].axis("off")
            continue

        img_np = np.array(img.resize((512, 512)))
        img_tensor = transform(img).unsqueeze(0).to(device) * 2 - 1

        with torch.no_grad():
            pred = model(img_tensor)
            pred_sigmoid = torch.sigmoid(pred)[0, 0].cpu().numpy()

        # Overlay
        pred_overlay = overlay_mask(np.mean(img_np, axis=2), pred_sigmoid)

        axs[row, 0].imshow(np.mean(img_np, axis=2), cmap="gray")
        axs[row, 0].set_title("GT: Input")
        axs[row, 1].imshow(np.array(mask.resize((512, 512))), cmap="gray")
        axs[row, 1].set_title("GT: Mask")
        axs[row, 2].imshow(pred_overlay, cmap="gray")
        axs[row, 2].set_title("Pred: Overlay")

        for col in range(3):
            axs[row, col].axis("off")
    
    plt.tight_layout()
    plt.savefig(f"grid_{batch}.png")
    plt.show()


### TODO:

train model for longer with smaller lr. maybe that will solve the problem of huge sizes. 